In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import xarray as xr
import numpy as np
import time as t
from multiprocess import Process, Manager

In [3]:
regions = ['reg_1','reg_2']

x_locations = np.arange(3)
y_locations = np.arange(3)

time = np.arange(3)

## time series data
ts_var = np.array([ [[[1, 1, 1] for i in range(3)] for i in range(3)],
                            
                    [[[1, 1, 1] for i in range(3)] for i in range(3)]
                   
                  ])

ts_var_da = xr.DataArray(ts_var, 
                        coords=[regions, y_locations, x_locations, time], 
                        dims=['regions', 'y', 'x','time'])

## values data
values_var = np.array([  [[1, 1, 1] for i in range(3)],

                         [[1, 1, 1] for i in range(3)]
                ])
values_var_da = xr.DataArray(values_var, 
                            coords=[regions, y_locations, x_locations], 
                            dims=['regions', 'y', 'x'])

ds = xr.Dataset({'ts_var': ts_var_da,
                'values_var': values_var_da})

In [12]:
n_types = 2

In [13]:
manager = Manager()
aggregated_dict = manager.dict()

In [18]:
def _aggregate_xr_ds(region, regional_ds):
    
    #prepare a resultant xr dataset
    time_steps = ds['time'].values
    types = [f'type_{i}' for i in range(n_types)] 

    data = np.zeros((len(time_steps), n_types))

    res_ts_var_da = xr.DataArray(data, [('time', time_steps),
                                        ('types', types)])

    data = np.zeros(len(types))

    res_values_var_da = xr.DataArray(data, [('types', types)])
        
    regional_ts_da = regional_ds['ts_var']
    regional_value_da = regional_ds['values_var']

    #Preprocess - stack and transpose dimensions 
    regional_ts_da = regional_ts_da.stack(x_y = ['x', 'y']) 
    regional_ts_da = regional_ts_da.transpose(transpose_coords= True) 

    regional_value_da = regional_value_da.stack(x_y = ['x', 'y'])
    regional_value_da = regional_value_da.transpose(transpose_coords= True)

    #STEP 5d. Aggregation
    ## values 
    res_values_var_da.loc[types[0]] = regional_value_da[dict(x_y=slice(None, 3))].sum()
    res_values_var_da.loc[types[1]] = regional_value_da[dict(x_y=slice(3, 9))].sum()
    ## ts 
    res_ts_var_da.loc[:, types[0]] = regional_ts_da[dict(x_y=slice(None, 3))].sum()
    res_ts_var_da.loc[:, types[1]] = regional_ts_da[dict(x_y=slice(3, 9))].sum()

    #Create resulting dataset 
    res_ds = xr.Dataset({"values": res_values_var_da,
                         "ts": res_ts_var_da}) 
    
    aggregated_dict.update({region : res_ds})

In [19]:
da_dict = dict(ds.groupby('regions'))

In [20]:


job = [Process(target=_aggregate_xr_ds, args=(key, value)) for key, value in da_dict.items()]
_ = [p.start() for p in job]
_ = [p.join() for p in job]

In [21]:
print(aggregated_dict)

{}
